# **ML-Pipeline**

ML Pipeline is a way to streamline the process of building and deploying machine learning models. 

It involves a series of steps that include data preprocessing, feature engineering, model training, and evaluation. 

The goal is to create a repeatable and efficient workflow that can be easily managed and scaled.

Pipelines help in automating the workflow of machine learning tasks, making it easier to manage and scale the process.

They allow for the encapsulation of multiple steps in a single object, which can be easily reused and modified.

Pipelines can also help in avoiding data leakage by ensuring that the same transformations are applied to both training and test data.

Pipelines also help in organizing the code and making it more readable.
They allow you to define a sequence of steps that can be executed in a single command.
This is particularly useful when you have multiple preprocessing steps or when you want to apply the same preprocessing steps to both training and testing data.

In [341]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

In [342]:
import pandas as pd
import numpy as np

num_items = 50000
genres = ['Rock', 'Metal', 'Bluegrass']

# 1. Generate clean features and target
followers = np.random.randint(50000, 5000000, num_items)
genre_col = np.random.choice(genres, num_items)

genre_score = pd.Series(genre_col).map({'Rock': 5, 'Metal': 7, 'Bluegrass': 2})
follower_score = (followers / 5000000) * 10
sold_out = (genre_score + follower_score > 10).astype(int)

# 2. Create the clean DataFrame
df = pd.DataFrame({
    'Genre': genre_col,
    'Social_media_followers': followers.astype(float), # Set as float for NaNs
    'Sold_out': sold_out
})

# 3. Inject NaNs directly into the DataFrame
# Randomly select a fraction of rows to set to NaN for each column
df.loc[df.sample(frac=0.11).index, 'Social_media_followers'] = np.nan
df.loc[df.sample(frac=0.22).index, 'Genre'] = np.nan

# 4. Show the final result
print("Dataset 1 Head:")
print(df.head())

print("\nTotal Missing Values:")
print(df.isnull().sum())

Dataset 1 Head:
       Genre  Social_media_followers  Sold_out
0        NaN               1140022.0         0
1       Rock                     NaN         0
2  Bluegrass               2013961.0         0
3  Bluegrass               4446759.0         1
4      Metal               4833486.0         1

Total Missing Values:
Genre                     11000
Social_media_followers     5500
Sold_out                      0
dtype: int64


In [343]:
df = df.drop(columns=['Genre'])
df.head()

,Social_media_followers,Sold_out
0,1140022.0,0
1,NaN,0
2,2013961.0,0
3,4446759.0,1
4,4833486.0,1


In [344]:
df = df.dropna(subset=['Social_media_followers', 'Sold_out'])

In [345]:
X = df.drop(['Sold_out'], axis=1)
y = df['Sold_out']

In [346]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)   

In [347]:
display(X_train.shape, y_train.shape)
display(X_test.shape, y_test.shape)

(35600, 1)

(35600,)

(8900, 1)

(8900,)

In [348]:
from sklearn.impute import SimpleImputer

# Create an imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

In [349]:
from sklearn.linear_model import LogisticRegression

logr = LogisticRegression()

In [350]:
pipe1 = make_pipeline(imputer, logr)
pipe1.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('logisticregression', LogisticRegression())])

In [351]:
print("Pipe Final Test 1: ", pipe1.score(X_test, y_test))

Pipe Final Test 1:  0.8155056179775281


In [352]:
print("Pipe Final Train 1: ", pipe1.score(X_train, y_train))

Pipe Final Train 1:  0.8173595505617978


In [353]:
# Display the statistics of the imputer
# This will show the mean values used to fill missing data
# This is useful to understand how the imputer has transformed the data
pipe1.named_steps.simpleimputer.statistics_

array([2527119.09530899])

In [354]:
# Display the coefficients of the logistic regression model
# This will show the importance of each feature in the model
# This is useful to understand how the model is making predictions
pipe1.named_steps.logisticregression.coef_

array([[1.64927478e-06]])

**Advanced Pipeline**

Implementing advanced pipeline with both numerical and categorical data.

In [355]:
import pandas as pd
import numpy as np

num_items = 50000
genres = ['Rock', 'Metal', 'Bluegrass']

# 1. Generate clean features and target
followers = np.random.randint(50000, 5000000, num_items)
genre_col = np.random.choice(genres, num_items)

genre_score = pd.Series(genre_col).map({'Rock': 5, 'Metal': 7, 'Bluegrass': 2})
follower_score = (followers / 5000000) * 10
sold_out = (genre_score + follower_score > 10).astype(int)

# 2. Create the clean DataFrame
df2 = pd.DataFrame({
    'Genre': genre_col,
    'Social_media_followers': followers.astype(float), # Set as float for NaNs
    'Sold_out': sold_out
})

# 3. Inject NaNs directly into the DataFrame
# Randomly select a fraction of rows to set to NaN for each column
df2.loc[df.sample(frac=0.11).index, 'Social_media_followers'] = np.nan
df2.loc[df.sample(frac=0.22).index, 'Genre'] = np.nan

# 4. Show the final result
print("Dataset 2 Head:")
print(df2.head())

print("\nTotal Missing Values:")
print(df2.isnull().sum())

Dataset 2 Head:
       Genre  Social_media_followers  Sold_out
0        NaN               1418107.0         0
1  Bluegrass                972438.0         0
2       Rock               2850984.0         1
3      Metal               1540391.0         1
4      Metal               3446546.0         1

Total Missing Values:
Genre                     9790
Social_media_followers    4895
Sold_out                     0
dtype: int64


In [356]:
df2 = df2.dropna(subset=['Social_media_followers', 'Sold_out'])

In [357]:
X2 = df2.iloc[:, 0:2]
y2 = df2.iloc[:, 2]

In [358]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=50)

In [359]:
num_col = ['Social_media_followers']
cat_col = ['Genre']

In [360]:
# Create pipelines for numerical and categorical data preprocessing
# The numerical pipeline will handle missing values and scale the data
# The categorical pipeline will handle missing values and apply one-hot encoding
# This allows us to preprocess different types of data in a consistent manner
# This is useful for building robust machine learning models that can handle various data types
num_pipeline = Pipeline(steps = [
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])

cat_pipeline = Pipeline(steps = [
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot-encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [361]:
from sklearn.compose import ColumnTransformer, make_column_transformer

# Create a column transformer to apply the numerical and categorical pipelines
# This allows us to apply different preprocessing steps to different columns in the dataset
# The `remainder='drop'` argument ensures that any columns not specified in the transformers are dropped from the final output
# This is useful for creating a clean and consistent dataset for training machine learning models
# It is also useful for building robust machine learning models that can handle various data types
# The `n_jobs=-1` argument allows the transformer to use all available CPU cores for parallel processing, speeding up the transformation process
col_transformer = ColumnTransformer(
    transformers = [
    ('num_pipeline', num_pipeline, num_col),
    ('cat_pipeline', cat_pipeline, cat_col)
    ],
    remainder='passthrough',  # Drop any columns not specified in the transformers
    n_jobs = -1  # Use all available CPU cores for parallel processing
)

In [362]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(max_depth=8, random_state=50)
pipe_final = make_pipeline(col_transformer, dtc)
pipe_final.fit(X2_train, y2_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['Social_media_followers']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot-encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Genre'])])),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(max_depth=8, random_state=50))])

In [363]:
print("Pipe Final Test 2: ", pipe_final.score(X2_test, y2_test))

Pipe Final Test 2:  0.9680744928500167


In [364]:
print("Pipe Final Train 2: ", pipe_final.score(X2_train, y2_train))

Pipe Final Train 2:  0.968628755126926


**Saving the Pipeline**

Saving the Pipeline is useful for later use, allowing us to reuse the trained model without retraining

In [365]:
import joblib

# Save the trained pipeline to a file
joblib.dump(pipe_final, 'music_genre_pipeline.joblib')

['music_genre_pipeline.joblib']

In [366]:
pipe_final_2 = joblib.load('music_genre_pipeline.joblib')